#### Tuning RNN Hyperparameters 

In [2]:
import sys

# Add path to the root folder
sys.path.append('../../../../../')
sys.path.append('../../../../features/prediction/')

import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import MinMaxScaler
from models.features.prediction.putils.formatter import create_sequences
from models.features.prediction.config.control import CONFIG
from models.features.prediction.config.path import BASE_DATASET_PATH
from models.features.prediction.manager import DataManager

In [3]:
class RNNHyperparameterTuner:
    def __init__(self, dataset, size_train, size_test):
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.dataset = dataset
        self.scaled_dataset = self.scaler.fit_transform(
            self.dataset.values.reshape(-1, 1)
        )
        self.scaled_training_dataset = self.scaled_dataset[:size_train]
        self.scaled_testing_dataset = self.scaled_dataset[
            size_train : size_train + size_test
        ]

    def train_model(self, config):
        # Group data for RNN
        X, y = create_sequences(
            self.scaled_training_dataset,
            config.get("n_past", 5),
            config.get("steps", 1),
        )

        # RNN Model
        model = Sequential()
        model.add(
            SimpleRNN(
                config.get("neurons", 50),
                activation="relu",
                input_shape=(X.shape[1], X.shape[2]),
                return_sequences=True,
            )
        )
        model.add(SimpleRNN(config.get("neurons", 50), activation="relu"))
        model.add(Dense(y.shape[1]))
        model.compile(
            optimizer=tf.keras.optimizers.Adam(
                learning_rate=config.get("learning_rate", 0.001)
            ),
            loss="mse",
        )

        # Train the model
        model.fit(
            X,
            y,
            epochs=config.get("epochs", 1),
            verbose=config.get("verbose", 0),
            batch_size=config.get("batch_size", 32),
            validation_split=config.get("validation_split", 0.2),
        )
        return model

    def evaluate_model(self, model, config):
        X_test, y_test = create_sequences(
            self.scaled_testing_dataset,
            config.get("n_past", 5),
            config.get("steps", 1),
        )
        # Evaluate the model on the test set
        test_loss = model.evaluate(X_test, y_test, verbose=0)
        return test_loss

    def random_search(self, n_iterations):
        best_score = float("inf")
        best_config = None

        for _ in range(n_iterations):
            config = {
                "n_past": random.choice([5, 10, 30, 50]),
                "steps": CONFIG["PREDICTION_STEPS"],
                "epochs": random.choice([10, 20, 50, 100]),
                "batch_size": random.choice([16, 32, 64]),
                "learning_rate": random.choice([0.001, 0.01, 0.1]),
                "neurons": random.choice([30, 50, 70]),
            }

            model = self.train_model(config)
            score = self.evaluate_model(model, config)

            if score < best_score:
                best_score, best_config = score, config
                print(f"New best score: {best_score} with config: {best_config}")

        print("Best Configuration:", best_config)

In [5]:
selected_feature = "cpu_usage"
size_train, size_test = 1000, 250
dataset = DataManager.LoadDataset("../../../../../" + BASE_DATASET_PATH)[selected_feature]

# Usage
tuner = RNNHyperparameterTuner(dataset, size_train, size_test)
tuner.random_search(n_iterations=10)

New best score: 0.0023586328607052565 with config: {'n_past': 30, 'steps': 5, 'epochs': 20, 'batch_size': 16, 'learning_rate': 0.1, 'neurons': 30}
New best score: 0.0010145725682377815 with config: {'n_past': 5, 'steps': 5, 'epochs': 20, 'batch_size': 16, 'learning_rate': 0.01, 'neurons': 70}
New best score: 0.0006197203765623271 with config: {'n_past': 30, 'steps': 5, 'epochs': 100, 'batch_size': 16, 'learning_rate': 0.01, 'neurons': 70}
Best Configuration: {'n_past': 30, 'steps': 5, 'epochs': 100, 'batch_size': 16, 'learning_rate': 0.01, 'neurons': 70}
